In [1]:
!pip install emoji -q


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install tensorflow==2.15.0

  Using cached tensorflow-2.15.0-cp311-cp311-macosx_10_15_x86_64.whl.metadata (4.2 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 MB 777.3 kB/s eta 0:00:000:0100:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
import numpy as np
import pandas as pd
import emoji
import re
import nltk
import pickle
import string
import tensorflow as tf
import tensorflow_hub as tf_hub
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import STOPWORDS

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ardiansyahputra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ardiansyahputra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ardiansyahputra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
print(tf.__version__)

2.15.0


In [5]:
data = pd.read_csv('cyberbullying_tweets.csv')
data

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [52]:
new_documents = ["Super happy with my cyborg body", 
"Check out this link: https://example.com @user #hashtag 😂",
"The shoutern seas are my domain"]
new_documents = pd.DataFrame(new_documents, columns=['text'])
new_documents

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,text
0,Super happy with my cyborg body
1,Check out this link: https://example.com @user...
2,The shoutern seas are my domain


In [53]:
# Update STOPWORDS
txt_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

STOPWORDS.update(['rt',
                  'mkr',
                  'didn',
                  'bc',
                  'n',
                  'm',
                  'im',
                  'll',
                  'y',
                  've',
                  'u',
                  'ur',
                  'don',
                  't',
                  's',
                  'gt',
                  'p',
                  'ta',
                  'pvp',
                  'manag',
                  'kat',
                  're',
                  'co',
                  'wa',
                  'https'])

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs, mentions, and non-alphanumeric characters
    txt_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(txt_cleaning_re, ' ', text)

    # Remove emojis
    text = emoji.replace_emoji(text, replace="")

    # Remove stopwords
    text = " ".join([word for word in text.split() if word not in STOPWORDS])

    # Remove /r, /n characters, non-utf characters, numbers, and punctuations
    text = text.replace('\r', '').replace('\n', ' ')
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    text = re.sub('[0-9]+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove additional stopwords
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])

    # Remove contractions
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    # Clean hashtags
    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    text = " ".join(word.strip() for word in re.split('#|_', text))

    # Filter special characters
    text = " ".join(['' if ('$' in word) | ('&' in word) else word for word in text.split()])

    # Remove multiple spaces
    text = re.sub("\s\s+" , " ", text)

    return text

In [54]:
# Checking function
test = 'Check out this link: https://example.com @user #hashtag 😂'
cleaned_text = clean_text(test)
print(cleaned_text)

check link user hashtag


In [55]:
new_documents['processed_text'] = new_documents['text'].apply(lambda x: clean_text(x))

In [56]:
new_documents

,text,processed_text
0,Super happy with my cyborg body,super happy cyborg body
1,Check out this link: https://example.com @user...,check link user hashtag
2,The shoutern seas are my domain,shoutern seas domain


In [57]:
# Lemmatizing
lemmatizer = WordNetLemmatizer()
def lemmatizer_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

new_documents['processed_text'] = new_documents['processed_text'].apply(lambda text: lemmatizer_words(text))

In [58]:
new_documents

,text,processed_text
0,Super happy with my cyborg body,super happy cyborg body
1,Check out this link: https://example.com @user...,check link user hashtag
2,The shoutern seas are my domain,shoutern sea domain


In [44]:
# Download the Embedding Layer

url = "https://www.kaggle.com/models/google/nnlm/TensorFlow2/tf2-preview-en-dim128-with-normalization/1"

hub_layer = tf_hub.KerasLayer(url, output_shape=[128], input_shape=[], dtype=tf.string)

In [45]:
# Load the saved model
model_lstm_2 = tf.keras.models.load_model('model_lstm_2.h5', custom_objects={'KerasLayer': hub_layer})

# Prediction

In [59]:
processed_texts = new_documents['processed_text']
processed_texts

0    super happy cyborg body
1    check link user hashtag
2        shoutern sea domain
Name: processed_text, dtype: object

In [60]:
y_pred_lstm_2 = model_lstm_2.predict(processed_texts)
y_pred_lstm_2


1/1 [==============================] - 0s 85ms/step


array([[0.0119455 , 0.02095619, 0.19604816, 0.40232334, 0.35906777,
        0.00965911],
       [0.00088618, 0.00228926, 0.02602725, 0.6620439 , 0.29411632,
        0.01463705],
       [0.0090423 , 0.00201403, 0.11807046, 0.5684629 , 0.27022704,
        0.03218323]], dtype=float32)

In [61]:
# Probability for each class

y_pred_df = pd.DataFrame(y_pred_lstm_2, columns=data['cyberbullying_type'].unique())
y_pred_df


,not_cyberbullying,gender,religion,other_cyberbullying,age,ethnicity
0,0.011946,0.020956,0.196048,0.402323,0.359068,0.009659
1,0.000886,0.002289,0.026027,0.662044,0.294116,0.014637
2,0.009042,0.002014,0.118070,0.568463,0.270227,0.032183


In [62]:
class_names = data['cyberbullying_type'].unique()
class_names

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [64]:
# Get class prediction

y_pred_class = np.argmax(y_pred_lstm_2[2])
y_pred_class_name = class_names[np.argmax(y_pred_lstm_2[2])]

print('Prediction - Class       : ', y_pred_class)
print('Prediction - Class Name  : ', y_pred_class_name)

Prediction - Class       :  3
Prediction - Class Name  :  other_cyberbullying
